In [3]:
# 라이브러리
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import random

# 파라미터 (최종 저장 경로 설정)
PARQUET_PATH = 'review_data_optimized.parquet'
MODEL_SAVE_PATH = 'final_best_hybrid_gemini_model.keras'

# 데이터 로드
df = pd.read_parquet(PARQUET_PATH)
print("데이터 로드 완료")

# 데이터 분할 및 전처리
df_processed = df[['user_id', 'business_id', 'stars', 'embedding']].copy()

# 데이터 7:1:2로 분할
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8  # 전체 데이터의 10%
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

# LabelEncoder를 사용하여 사용자 ID와 비즈니스 ID 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

train_df.loc[:, 'user_encoded'] = user_encoder.fit_transform(train_df['user_id'])
train_df.loc[:, 'business_encoded'] = business_encoder.fit_transform(train_df['business_id'])

user_mapping = {label: i for i, label in enumerate(user_encoder.classes_)}
business_mapping = {label: i for i, label in enumerate(business_encoder.classes_)}

val_df.loc[:, 'user_encoded'] = val_df['user_id'].map(user_mapping).fillna(-1).astype(int)
val_df.loc[:, 'business_encoded'] = val_df['business_id'].map(business_mapping).fillna(-1).astype(int)
test_df.loc[:, 'user_encoded'] = test_df['user_id'].map(user_mapping).fillna(-1).astype(int)
test_df.loc[:, 'business_encoded'] = test_df['business_id'].map(business_mapping).fillna(-1).astype(int)

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

train_embeddings = np.array(train_df['embedding'].tolist())
val_embeddings = np.array(val_df['embedding'].tolist())
test_embeddings = np.array(test_df['embedding'].tolist())

gemini_embedding_dim = len(train_df['embedding'].iloc[0]) if not train_df.empty else 3072
print(f"제미니 임베딩 차원: {gemini_embedding_dim}")

데이터 로드 완료
전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)
제미니 임베딩 차원: 3072


In [4]:
def build_hybrid_gemini_model(num_users, num_businesses, user_embedding_dim, business_embedding_dim,
                             gemini_embedding_dim, user_biz_mlp_dims, gemini_mlp_dims, final_mlp_dims):
    
    # 사용자-비즈니스 상호작용 모듈
    user_input = keras.Input(shape=(1,), name='user_id')
    business_input = keras.Input(shape=(1,), name='business_id')
    user_embedding = layers.Embedding(num_users, user_embedding_dim, name='user_embedding')(user_input)
    user_vec = layers.Flatten()(user_embedding)
    business_embedding = layers.Embedding(num_businesses, business_embedding_dim, name='business_embedding')(business_input)
    business_vec = layers.Flatten()(business_embedding)
    combined_vec = layers.concatenate([user_vec, business_vec], axis=1)
    interaction_features = combined_vec
    for dim in user_biz_mlp_dims:
        interaction_features = layers.Dense(dim, activation='relu')(interaction_features)

    # 제미니 임베딩 모듈
    gemini_input = keras.Input(shape=(gemini_embedding_dim,), name='gemini_embedding')
    gemini_features = gemini_input
    for dim in gemini_mlp_dims:
        gemini_features = layers.Dense(dim, activation='relu')(gemini_features)
    
    # 최종 예측 모듈
    final_combined_features = layers.concatenate([interaction_features, gemini_features], axis=1)
    predicted_rating = final_combined_features
    for dim in final_mlp_dims:
        predicted_rating = layers.Dense(dim, activation='relu')(predicted_rating)
    predicted_rating = layers.Dense(1, activation='linear', name='output_rating')(predicted_rating)
    
    model = models.Model(inputs=[user_input, business_input, gemini_input], outputs=predicted_rating)
    return model

In [5]:
# --- 최적 파라미터 탐색 공간 정의 ---
param_distributions = {
    'user_embedding_dim': [32, 64, 128],
    'business_embedding_dim': [32, 64, 128],
    'gemini_mlp_dims': [
        [1536, 768],
        [1536, 768, 384],
        [1536, 768, 384, 192]
    ],
    'user_biz_mlp_dims': [
        [64, 32],
        [128, 64],
        [256, 128]
    ],
    'final_mlp_dims': [
        [64, 32],
        [128, 64],
        [256, 128]
    ],
    'learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001],
    'batch_size': [64, 128, 256]
}

# --- 랜덤 서치 실행 설정 ---
num_searches = 10 

best_val_rmse = float('inf')
best_params = None
best_model = None

# --- 랜덤 서치 루프 ---
print("\n" + "="*50)
print(f"랜덤 서치를 통해 최적 하이퍼파라미터를 찾습니다. (탐색 횟수: {num_searches})")
print("="*50)

for i in range(num_searches):
    print(f"\n--- 랜덤 서치 {i+1}/{num_searches} 시작 ---")

    # 하이퍼파라미터 무작위 샘플링
    current_params = {key: random.choice(values) for key, values in param_distributions.items()}
    print(f"샘플링된 파라미터: {current_params}")
    
    # 모델 구축 및 컴파일
    model = build_hybrid_gemini_model(
        num_users, num_businesses,
        current_params['user_embedding_dim'], current_params['business_embedding_dim'],
        gemini_embedding_dim,
        current_params['user_biz_mlp_dims'], current_params['gemini_mlp_dims'],
        current_params['final_mlp_dims'])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=current_params['learning_rate']),
                    loss='mse',
                    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae'])
    
    # EarlyStopping 콜백
    early_stopping_callback = callbacks.EarlyStopping(
        monitor='val_rmse', patience=5, min_delta=0.0005, mode='min', restore_best_weights=True)
        
    # 모델 학습 (verbose=0으로 설정하여 출력 간소화)
    history = model.fit(
        {'user_id': train_df['user_encoded'],
         'business_id': train_df['business_encoded'],
         'gemini_embedding': train_embeddings},
        train_df['stars'],
        batch_size=current_params['batch_size'],
        epochs=50,
        validation_data=(
            {'user_id': val_df['user_encoded'],
             'business_id': val_df['business_encoded'],
             'gemini_embedding': val_embeddings},
            val_df['stars']
        ),
        callbacks=[early_stopping_callback],
        verbose=0)

    current_val_rmse = min(history.history['val_rmse'])
    print(f" -> 검증 RMSE: {current_val_rmse:.4f}")
    
    # 최적 파라미터 업데이트
    if current_val_rmse < best_val_rmse:
        best_val_rmse = current_val_rmse
        best_params = current_params
        best_model = model
        print(" -> 최적 모델 업데이트!")

print("\n" + "="*50)
print(f"최종 최적 파라미터: {best_params}")
print(f"최소 검증 RMSE: {best_val_rmse:.4f}")
print("="*50)


랜덤 서치를 통해 최적 하이퍼파라미터를 찾습니다. (탐색 횟수: 10)

--- 랜덤 서치 1/10 시작 ---
샘플링된 파라미터: {'user_embedding_dim': 128, 'business_embedding_dim': 64, 'gemini_mlp_dims': [1536, 768, 384, 192], 'user_biz_mlp_dims': [64, 32], 'final_mlp_dims': [128, 64], 'learning_rate': 0.005, 'batch_size': 256}
 -> 검증 RMSE: 0.4755
 -> 최적 모델 업데이트!

--- 랜덤 서치 2/10 시작 ---
샘플링된 파라미터: {'user_embedding_dim': 32, 'business_embedding_dim': 128, 'gemini_mlp_dims': [1536, 768], 'user_biz_mlp_dims': [128, 64], 'final_mlp_dims': [128, 64], 'learning_rate': 0.005, 'batch_size': 256}
 -> 검증 RMSE: 0.4769

--- 랜덤 서치 3/10 시작 ---
샘플링된 파라미터: {'user_embedding_dim': 64, 'business_embedding_dim': 32, 'gemini_mlp_dims': [1536, 768], 'user_biz_mlp_dims': [256, 128], 'final_mlp_dims': [256, 128], 'learning_rate': 0.0001, 'batch_size': 256}
 -> 검증 RMSE: 0.4707
 -> 최적 모델 업데이트!

--- 랜덤 서치 4/10 시작 ---
샘플링된 파라미터: {'user_embedding_dim': 128, 'business_embedding_dim': 128, 'gemini_mlp_dims': [1536, 768, 384, 192], 'user_biz_mlp_dims': [64, 32], 'final

In [6]:
# 최적 모델 저장 및 최종 평가
if best_model:
    best_model.save(MODEL_SAVE_PATH)
    print(f"최적 모델이 {MODEL_SAVE_PATH}에 저장되었습니다.")
    final_model = best_model
else:
    print(f"최적 모델을 찾을 수 없습니다. 기본 모델을 로드합니다.")
    # 랜덤 서치가 실패했을 경우 대비 (일반적으로 발생하지 않음)
    final_model = keras.models.load_model(MODEL_SAVE_PATH)

# --- 모델 평가 ---
print("\n" + "="*50)
print("모델 평가 시작")
print("="*50)

test_predictions = final_model.predict(
    {'user_id': test_df['user_encoded'],
     'business_id': test_df['business_encoded'],
     'gemini_embedding': test_embeddings}).flatten()

true_ratings = test_df['stars'].values
mse = mean_squared_error(true_ratings, test_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_ratings, test_predictions)
mape = mean_absolute_percentage_error(true_ratings, test_predictions)

print(f"최적 파라미터: {best_params}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}")

최적 모델이 final_best_hybrid_gemini_model.keras에 저장되었습니다.

모델 평가 시작
2799/2799 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
최적 파라미터: {'user_embedding_dim': 128, 'business_embedding_dim': 32, 'gemini_mlp_dims': [1536, 768, 384, 192], 'user_biz_mlp_dims': [128, 64], 'final_mlp_dims': [64, 32], 'learning_rate': 0.0001, 'batch_size': 256}
Mean Squared Error (MSE): 0.2182
Root Mean Squared Error (RMSE): 0.4671
Mean Absolute Error (MAE): 0.3547
Mean Absolute Percentage Error (MAPE): 0.1143
